In [1]:
from scipy.integrate import ode
from scipy.interpolate import interp1d
import pandas as pd
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
import time
from scipy.optimize import minimize, approx_fprime, minimize_scalar

In [2]:
import matplotlib as mpl
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.autolayout'] = True 
def utkarshGrid():
    plt.minorticks_on()
    plt.grid(color='grey',
             which='minor',
             linestyle=":",
             linewidth='0.1',
             )
    plt.grid(color='black',
             which='major',
             linestyle=":",
             linewidth='0.1',
             )

In [6]:
G = 6.67259e-8 # dyne cm2 g-2
c = 2.99792458e10 # cms-1
n_den = 2.3e14
km2cm = 1e5
msun = 1.98847e33

def lamba_metric(M, R):
    return -np.log((1-2*G*M/(c**2*R)))

def df_to_ep(df):
    e_den = df.energy_densityc2
    p_arr = df.pressurec2
    p = p_arr[p_arr > 1e8]
    e_den = e_den[p_arr > 1e8]
    e_den = e_den*((2.99792458e10)**2)
    pressure = p*((2.99792458e10)**2)
    return e_den.to_numpy(), pressure.to_numpy()
def get_ep(e, p):
    f_e_smooth = interp1d(p, e, fill_value= "extrapolate", kind = "cubic")
    return f_e_smooth
def get_pe(p, e):
    f_e_smooth = interp1d(e, p, fill_value=(0, 0), kind = "cubic", bounds_error=True)
    return f_e_smooth


path = "2205.02081/SLY230A.csv"
df = pd.read_csv(path)
e, p = df_to_ep(df)
p_max = max(p)
p_min = max(1*(c**2), min(p)) 
EOS = get_ep(e, p)

In [ ]:
def _dMdr(r, e):
    return 4 * np.pi * r**2 * (e/(c**2))
def _b(r, M):
    return (G*M)/((c**2)*r)
def _dvdr(r, Q, lamda):
    return (2/r) * np.exp(lamda) * Q
def _Q(r, P, M):
    frac = (4 * np.pi * G * (r**2)*P)/(c**4)
    return _b(r, M) + frac
def _dPdr(r, P, M, e):    
    num = (M+4*np.pi*(r**3)*P/(c**2))
    dem = r * (r-2*G*M/(c**2))
    return -1 * (e+P) * G/(c**2) * num/dem
def dedP_helper(p, e):
    return np.gradient(e, p), e
def dedP(p, e):
    dedp_helper, e_arr = dedP_helper(p, e)
    return interp1d(e_arr, dedp_helper, fill_value="extrapolate", kind = "cubic")
def drhodP(e, dedp = dedP(p, e)):
    return (c**-2) * dedp(e)